In [ ]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
# os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
# import torch
# print('Current cuda device:', torch.cuda.current_device())
# print('Count of using GPUs:', torch.cuda.device_count())

In [1]:
import os
import random
import time
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import ElectraModel, ElectraTokenizer, ElectraConfig

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

/opt/tljh/user/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-20 09:57:20.280800: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 09:57:22.314703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
STATE_DICT_PATH = 'training_results/v4.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

labels = ['[PAD]', 'E_B', 'E_I', 'O']
num_labels = len(labels)
id2label = {k: v for k, v in enumerate(labels)}
label2id = {v: k for k, v in id2label.items()}

max_len = 256
batch_size = 32

## Load Model and Tokenizer

In [3]:
config = ElectraConfig.from_pretrained('config')
bert = ElectraModel(config)
tokenizer = ElectraTokenizer.from_pretrained('tokenizer')

In [4]:
class BERTPoSTagger(nn.Module):
    def __init__(self, bert, output_dim, dropout):
        super().__init__()
        
        self.bert = bert
        self.dropout = nn.Dropout(dropout)
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text, mask):
        embedded = self.dropout(self.bert(text, mask)[0])
        predictions = self.fc(self.dropout(embedded))
        return predictions

In [5]:
OUTPUT_DIM = num_labels
DROPOUT = 0.25

model = BERTPoSTagger(bert, OUTPUT_DIM, DROPOUT)
model.bert.resize_token_embeddings(len(tokenizer))

NGPU = torch.cuda.device_count()
if NGPU > 1:
    model = torch.nn.DataParallel(model, device_ids=list(range(NGPU)))

model.to(device)
model.load_state_dict(torch.load(STATE_DICT_PATH))

<All keys matched successfully>

## Load Data

In [6]:
data = pd.read_pickle('data/preprocessed.pkl')

data['tokens'] = data.tokens.apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
data['labels'] = data.labels.apply(lambda x: ['O'] + x + ['O'])

data['tokens'] = data.tokens.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))
data['labels'] = data.labels.apply(lambda x: x + ['[PAD]'] * (max_len - len(x)))

tokens_lst = data.tokens.to_list()
labels_lst = data.labels.to_list()

X_train, X_eval, y_train, y_eval = train_test_split(tokens_lst, 
                                                    labels_lst, 
                                                    test_size=0.2, shuffle=True, random_state=42)

train_data = []
for tokens, labels in zip(X_train, y_train):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)

    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    train_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])
    
eval_data = []
for tokens, labels in zip(X_eval, y_eval):
    length = tokens.index('[PAD]')
    mask = [1] * length + [0] * (max_len - length)
    
    label_ids = []
    for label in labels:
        label_ids.append(label2id[label])
        
    eval_data.append([tokenizer.convert_tokens_to_ids(tokens), mask, label_ids])
    
class TaggerDataset(Dataset): 
    def __init__(self, data):
        self.data = data
    
    def __len__(self): 
        return len(self.data)

    def __getitem__(self, idx):
        input_ids = self.data[idx][0]
        mask = self.data[idx][1]
        label_ids = self.data[idx][2]
        return (torch.LongTensor(input_ids), torch.LongTensor(mask), torch.LongTensor(label_ids))
    
train_dataset = TaggerDataset(train_data)
eval_dataset = TaggerDataset(eval_data)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
eval_loader = DataLoader(eval_dataset, batch_size = batch_size, shuffle = False)

## Functions

In [7]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])

    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

In [8]:
def categorical_f1(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = -1, keepdim = True) # get the index of the max probability
    non_pad_elements = torch.nonzero(y != tag_pad_idx)
    max_preds_no_pad = max_preds[non_pad_elements].squeeze(1).detach().cpu()
    y_no_pad = y[non_pad_elements].detach().cpu()
    
    f1_macro = f1_score(y_no_pad, max_preds_no_pad, average='macro')
    f1_micro = f1_score(y_no_pad, max_preds_no_pad, average='micro')    
    
    return f1_macro, f1_micro

In [9]:
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [10]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    model.eval()
    epoch_loss = 0
    epoch_acc = 0
    predictions_set = None
    tags_set = None
    with torch.no_grad():
        for batch in iterator:
            text = batch[0].to(device)
            mask = batch[1].to(device)
            tags = batch[2].to(device)
            
            predictions = model(text, mask)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            if predictions_set == None:
                predictions_set = predictions
                tags_set = tags
            else:
                predictions_set = torch.cat([predictions_set, predictions], dim=0)
                tags_set = torch.cat([tags_set, tags], dim=0)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        f1_macro, f1_micro = categorical_f1(predictions_set, tags_set, tag_pad_idx)
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator), f1_macro, f1_micro

## Inferrence

In [11]:
for batch in eval_loader:
    input_ids, mask, label_ids = batch
    input_ids, mask, label_ids = input_ids.to(device), mask.to(device), label_ids.to(device)
    
    output = model(input_ids, mask).argmax(-1)
    break

In [12]:
preds = output.tolist()
preds = {'preds': preds}
preds = pd.DataFrame(preds)

In [13]:
preds2labels = preds.preds.apply(lambda lst: list(map(lambda x: id2label[x], lst)))

In [14]:
inputs = input_ids.tolist()
inputs = {'inputs': inputs}
inputs = pd.DataFrame(inputs)

In [15]:
inputs2labels = inputs.inputs.apply(lambda lst: list(map(lambda x: tokenizer.convert_ids_to_tokens(x), lst)))

In [16]:
for ins, outs in zip(inputs2labels, preds2labels):
    idx = ins.index('[PAD]')
    ins, outs = ins[:idx], outs[:idx]
    for x, y in zip(ins, outs):
        print(x, y)
    print()

[CLS] O
2020 O
##년 O
8 O
##월 O
1 O
4 O
일 O
오후 O
1 O
##시 O
5 O
##분 O
##경 O
부산 O
진구 O
##의 O
한 O
건물 O
주차타워 O
에서 O
리스트 O
수리작업 O
중 O
이 O
##던 O
근로자 O
##가 O
추락 O
##하여 O
사망 E_B
##하 O
##였 O
##다 O
. O
리프트 E_B
수리 E_I
작업 E_I
중 O
물건 E_B
##을 E_I
가지 E_I
##러 E_I
아래 E_I
층 E_I
##으로 E_I
이 E_I
동 E_I
하 O
##다가 O
추락 E_B
##한 O
것 O
##으로 O
추정 O
##하 O
##고 O
사고 O
조사 O
중 O
에 O
있 O
##다 O
. O
[SEP] O

[CLS] O
2022 O
##년 O
3 O
##월 O
22 O
##일 O
오전 O
11 O
##시 O
30 O
##분 O
##경 O
경남 O
거제 O
소재 O
사업장 O
##에 O
##서 O
선박 O
블록 O
##을 O
취 O
##부 O
##하 O
##기 O
위해 O
블록 E_B
사이 E_I
##에 E_I
##서 E_I
작업 E_I
하 O
##던 O
중 O
선박 E_B
블록 E_I
##이 E_I
움직여 E_I
선박 O
블록 O
사이 O
##에 O
재해자 E_B
가 E_I
끼 E_I
##이 E_I
##는 E_I
사고 O
##가 O
발생 O
##하 O
##였 O
##다 O
. O
이 O
사고 O
##로 O
작업자 O
가 O
사망 E_B
##하 O
##였 O
##다 O
. O
[SEP] O

[CLS] O
2021 O
##년 O
12 O
##월 O
10 O
##일 O
오전 O
10 O
##시 O
##경 O
강원 O
횡성군 O
의 O
공장 O
신축공사 O
장에 O
##서 O
작업자 O
가 O
20 O
##m O
아래 O
로 O
추락 E_B
##하 O
##는 O
사고 O
##가 O
발생 O
##하 O
##였 O
##다 O
. O
이 O
사고 O
##로 O
작업자 O
가 O
사망 E_B
##하 O
##였 O
##